# 🚀 GEAK-Agent Tutorial
### Generating Optimized GPU Kernels with LLM Agents

---

This notebook demonstrates **GEAK-Agent** - a multi-agent LLM framework that automatically generates and optimizes [Triton](https://github.com/openai/triton) GPU kernels.

| Step | What Happens |
|:----:|-------------|
| 1️⃣ | **Setup** - Configure API keys |
| 2️⃣ | **Initialize** - Load kernels and create agent |
| 3️⃣ | **Run** - Generate and optimize kernels |
| 4️⃣ | **Results** - View generated code and metrics |

---
## 1️⃣ Setup

In [2]:
""" Please do this for setup
git clone https://github.com/AMD-AGI/GEAK-agent.git
cd GEAK-agent
python3 -m pip install -r requirements.txt
"""

' Please do this for setup\ngit clone https://github.com/AMD-AGI/GEAK-agent.git\ncd GEAK-agent\npython3 -m pip install -r requirements.txt\n'

In [1]:
# Initialize environment
import os, sys

# Robust path detection
current_path = os.getcwd()
if 'GEAK-agent' in current_path:
    # We are somewhere inside GEAK-agent
    while os.path.basename(current_path) != 'GEAK-agent':
        current_path = os.path.dirname(current_path)
    GEAK_DIR = current_path
elif os.path.exists('GEAK-agent'):
    # We are at the parent of GEAK-agent
    GEAK_DIR = os.path.abspath('GEAK-agent')
else:
    # Fallback: assume parent of current dir is GEAK-agent root if we are in tutorial
    GEAK_DIR = os.path.dirname(current_path)

SRC_DIR = os.path.join(GEAK_DIR, 'src')
TUTORIAL_DIR = os.path.join(GEAK_DIR, 'tutorial')

# Add paths
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)
if TUTORIAL_DIR not in sys.path:
    sys.path.insert(0, TUTORIAL_DIR)

from tutorial_utils import setup_environment, print_header
# Re-run setup_environment to ensure everything is consistent
_, _, CORPUS_PATH, TutorialDataloader = setup_environment()
print_header('✓ GEAK-Agent Tutorial Ready')

╔════════════════════════════════════════════════════════════╗
║ ✓ GEAK-Agent Tutorial Ready                                ║
╚════════════════════════════════════════════════════════════╝


In [8]:
# ═══════════════════════════════════════════════════════════════
#                       🔑 API CONFIGURATION
# ═══════════════════════════════════════════════════════════════

USE_AMD_API = False                             # Set False for OpenAI
AMD_API_KEY = 'efda07b64df94bef99096a9f1e3669a7'                # Your AMD API key
OPENAI_API_KEY = 'efda07b64df94bef99096a9f1e3669a7'              # Your OpenAI key

# ───────────────────────────────────────────────────────────────
API_KEY = AMD_API_KEY if USE_AMD_API else OPENAI_API_KEY
MODEL_ID = 'claude-sonnet-4' if USE_AMD_API else 'claude-sonnet-4'
print(f'✓ API: {"AMD" if USE_AMD_API else "OpenAI"} | Model: {MODEL_ID}')

✓ API: OpenAI | Model: claude-sonnet-4


---
## 2️⃣ Initialize Dataset & Agent

In [10]:
from models.Claude import ClaudeModel
from models.OpenAI import OpenAIModel
from agents.GaAgent import GaAgent

# Load kernels
dataset = TutorialDataloader(
    kernel_names=['embedding_triton_kernel.py'],
    corpus_path=CORPUS_PATH
)

# Initialize model & agent
model = OpenAIModel(api_key=API_KEY, model_id=MODEL_ID)

agent = GaAgent(model=model, dataset=dataset, corpus_path=CORPUS_PATH, descendant_num=3)

print(f'✓ Loaded {len(dataset)} kernels | Agent ready with {len(agent.memories)} tasks')

✓ Loaded 1 kernels | Agent ready with 1 tasks


In [11]:
# Preview kernel instructions (optional)
from tutorial_utils import display_kernel_info
display_kernel_info(dataset.problem_states)


┌──────────────────────────────────────────────────────────────────────┐
│ 📄 embedding_triton_kernel.py                                       │
├──────────────────────────────────────────────────────────────────────┤
│ You are a expert in writing Triton operators for efficient GPU progr │
│ amming. Use triton language write a kernel and wrapper according fol │
│ lowing instruction.             The Triton-accelerated function embe │
│ dding_kernel is specialized for extracting and storing embedding vec │
│ tors from a weight matrix for a sequence of token IDs. It uses progr │
│ am IDs to determine processing offsets and handles iteration         │
│ ...[truncated]                                                       │
├──────────────────────────────────────────────────────────────────────┤
│ ✓ Has reference code: True                                          │
│ ✓ Has test code: True                                               │
└─────────────────────────────────────────────────────

---
## 3️⃣ Run GEAK-Agent

Each iteration: **Generate** → **Test** → **Benchmark** → **Reflect**

> ⏱️ Takes a few minutes per iteration

In [12]:
from tutorial_utils import print_config

OUTPUT_DIR = os.path.join(TUTORIAL_DIR, 'outputs')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ═══════════════════════════════════════════════════════════════
#                       ⚙️ CONFIGURATION
# ═══════════════════════════════════════════════════════════════
CONFIG = {
    'iteration_num': 6,       # Optimization iterations
    'temperature': 1.0,       # LLM creativity (0.0-2.0)
    'descendant_num': 3,      # Candidates per iteration
    'ancestor_num': 5,        # Reference solutions
    'gpu_id': 0,              # GPU device
    'target_gpu': 'MI300X',   # Target AMD GPU
}
print_config(CONFIG, '🚀 Running GEAK-Agent')


════════════════════════════════════════════════════════════
 🚀 Running GEAK-Agent
════════════════════════════════════════════════════════════
  • iteration_num: 6
  • temperature: 1.0
  • descendant_num: 3
  • ancestor_num: 5
  • gpu_id: 0
  • target_gpu: MI300X
════════════════════════════════════════════════════════════


In [ ]:
# Run the agent
agent.run(
    output_path=os.path.join(OUTPUT_DIR, 'tutorial_results.jsonl'),
    multi_thread=False,
    **CONFIG,
    start_iter=0, descendant_debug=0, profiling=False, start_idx=0
)
print('\n✅ Complete!')

2025-12-11 20:52:12.024 | INFO     | agents.GaAgent:run:141 - 
=== Iteration 0 ===
2025-12-11 20:52:12.025 | INFO     | agents.GaAgent:run:150 - 
generate solution
  0%|          | 0/1 [00:00<?, ?it/s]2025-12-11 20:52:12.025 | INFO     | agents.GaAgent:run:150 - 
generate solution
  0%|          | 0/1 [00:00<?, ?it/s]2025-12-11 20:52:27.382 | INFO     | agents.GaAgent:call_llm_code:444 - failed to call LLM: RetryError[<Future at 0x7f8b466869b0 state=finished raised APIConnectionError>]
2025-12-11 20:52:27.383 | INFO     | agents.GaAgent:generate_solution:350 - failed to call LLM for embedding_triton_kernel.py
2025-12-11 20:52:27.382 | INFO     | agents.GaAgent:call_llm_code:444 - failed to call LLM: RetryError[<Future at 0x7f8b466869b0 state=finished raised APIConnectionError>]
2025-12-11 20:52:27.383 | INFO     | agents.GaAgent:generate_solution:350 - failed to call LLM for embedding_triton_kernel.py
2025-12-11 20:52:43.519 | INFO     | agents.GaAgent:call_llm_code:444 - failed to cal

---
## 4️⃣ View Results

In [ ]:
import openai
import os

try:
    user = os.getlogin()
except Exception:
    user = os.environ.get("USER", "unknown_user")
 
client = openai.OpenAI(
    base_url="https://llm-api.amd.com/OnPrem",
    api_key="dummy",
    default_headers={
        "Ocp-Apim-Subscription-Key": "efda07b64df94bef99096a9f1e3669a7",
        "user": user
    }
)
 
response = client.chat.completions.create(
    model="GPT-oss-20B",
    max_completion_tokens=200,
    temperature=0.7,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "How does AI work?"}
    ]
)
 
print(response)

OSError: [Errno 6] No such device or address

In [8]:
from tutorial_utils import load_results, display_results_summary, display_generated_code

results, iteration = load_results(OUTPUT_DIR)
display_results_summary(results, iteration)


╔══════════════════════════════════════════════════════════════════════╗
║ 📊 GEAK-Agent Results Summary                                         ║
║ Iteration: 5                                                         ║
╠══════════════════════════════════════════════════════════════════════╣
║ Kernel              │ Call │ Exec │ Perf │ Speedup                   ║
╠──────────────────────────────────────────────────────────────────────╣
║ embedding_triton_k │  ✗   │  ✓   │  ✗   │ 4.3293x                    ║
╚══════════════════════════════════════════════════════════════════════╝

 Legend: ✓ = Passed, ✗ = Failed
 Speedup: reference_time / generated_time (>1.0 = faster)


In [9]:
# View generated code
display_generated_code(results)


┌──────────────────────────────────────────────────────────────────────┐
│ 🔧 embedding_triton_kernel.py - ✓ Correct                             │
└──────────────────────────────────────────────────────────────────────┘
import torch
import triton
import triton.language as tl

@triton.jit
def embedding_kernel(
    weight,
    input_ids,
    out,
    vob_start_id,
    vob_end_id,
    stride_weight_seq,
    stride_out_seq,
    n_ctx,
    hiden_size: tl.constexpr,
    BLOCK_DMODEL: tl.constexpr,
    BLOCK_N: tl.constexpr,
    BLOCK_NN: tl.constexpr,
):
    # Get program IDs for parallel processing
    pid_seq = tl.program_id(0)
    pid_dim = tl.program_id(1)
    
    # Calculate sequence and dimension start offsets
    seq_start = pid_seq * BLOCK_N
    dim_start = pid_dim * BLOCK_DMODEL
    
    # Create offset ranges with wavefront alignment
    seq_offsets = seq_start + tl.arange(0, BLOCK_N)
    dim_offsets = dim_start + tl.arange(0, BLOCK_DMODEL)
    
    # Efficient boundary checking
 

---
## 📚 Reference

| Metric | Meaning |
|--------|--------|
| **pass_call** | Code runs without errors |
| **pass_exe** | Output matches reference |
| **speedup** | `ref_time/gen_time` (>1 = faster) |

**Tips:** More iterations → better results • Higher temperature → more diversity

---
🎉 **Done!** See [GEAK-Agent](https://github.com/AMD-AGI/GEAK-agent) for more.